In [1]:
# Code in this notebook heavily pulled from https://github.com/Xilinx/finn/blob/main/notebooks/end2end_example/bnn-pynq/tfc_end2end_example.ipynb

In [6]:
import finn
from finn.util.visualization import showSrc, showInNetron
from finn.util.basic import make_build_dir
import os
import onnx
file_path = str('../workspace/jh_fpga_amr/src/py/models')
file_name = str('/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx')
onnx_model = onnx.load(file_path + file_name)

In [7]:
from finn.util.visualization import showInNetron
showInNetron(file_path + file_name)

Serving '../workspace/jh_fpga_amr/src/py/models/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [8]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
model = ModelWrapper(file_path + file_name)
model = model.transform(ConvertQONNXtoFINN())

In [9]:
from finn.transformation.streamline import Streamline
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors

model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(Streamline())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model.save(build_dir + "/end2end_cnv_w1a1_streamlined.onnx")

IndexError: list index (1) out of range

In [5]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.util.visualization import showSrc

model = ModelWrapper(onnx_model)
showSrc(ModelWrapper.analysis)

    def analysis(self, analysis_fxn):
        """Runs given anaylsis_fxn on this model and return resulting dict."""
        return analysis_fxn(self)



In [6]:
import os
os.makedirs(os.path.dirname(file_path+'/conv'+file_name), exist_ok=True)
model.save(file_path+'/conv'+file_name)
showInNetron(file_path+'/conv'+file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/conv/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [7]:
## Tidy up transformations.
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

os.makedirs(os.path.dirname(file_path+'/transform'+file_name), exist_ok=True)
model.save(file_path+'/transform'+file_name)
showInNetron(file_path+'/transform'+file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/transform/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [8]:
## Preparing to convert to hardware layers.
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw

model = ModelWrapper(file_path+'/transform'+file_name)
model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
# TopK to LabelSelect.
model = model.transform(to_hw.InferLabelSelectLayer())
# Input quantization (if any) to standalone thresholding.
model = model.transform(to_hw.InferThresholdingLayer())

os.makedirs(os.path.dirname(file_path+'/hw_layers'+file_name), exist_ok=True)
model.save(file_path+'/hw_layers'+file_name)
showInNetron(file_path+'/hw_layers'+file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/hw_layers/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [9]:
## Separating HLS layers into another model.
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

parent_model = model.transform(CreateDataflowPartition())
os.makedirs(os.path.dirname(file_path+'/dataflow_parent'+file_name), exist_ok=True)
parent_model.save(file_path+'/dataflow_parent'+file_name)
showInNetron(file_path+'/dataflow_parent'+file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/dataflow_parent/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [10]:
# Change this if you have a different PYNQ board, see list above.
from finn.util.basic import pynq_part_map
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [11]:
## Each abstaction layer converted to HLS variant.
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
model = model.transform(SpecializeLayers(fpga_part))

os.makedirs(os.path.dirname(file_path+'/specialize_layers_hls'+file_name), exist_ok=True)
model.save(file_path+'/specialize_layers_hls'+file_name)
showInNetron(file_path+'/specialize_layers_hls'+file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/specialize_layers_hls/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [12]:
### Hoping to not have to use this at all.
## In this notebook we are setting the folding factors and FIFO depths manually but it is possible to use FINN transformations for this (SetFolding and InsertAndSetFIFODepths).

#fc_layers = model.get_nodes_by_op_type("MVAU_hls")
## (PE, SIMD, in_fifo_depth, out_fifo_depth, ramstyle) for each layer
#config = [
#    (16, 49, [16], [64], "block"),
#    (8, 8, [64], [64], "auto"),
#    (8, 8, [64], [64], "auto"),
#    (10, 8, [64], [10], "distributed"),
#]
#for fcl, (pe, simd, ififo, ofifo, ramstyle) in zip(fc_layers, config):
#    fcl_inst = getCustomOp(fcl)
#    fcl_inst.set_nodeattr("PE", pe)
#    fcl_inst.set_nodeattr("SIMD", simd)
#    fcl_inst.set_nodeattr("inFIFODepths", ififo)
#    fcl_inst.set_nodeattr("outFIFODepths", ofifo)
#    fcl_inst.set_nodeattr("ram_style", ramstyle)
    
## Set parallelism for input quantizer to be same as first layer's SIMD.
#inp_qnt_node = model.get_nodes_by_op_type("Thresholding_hls")[0]
#inp_qnt = getCustomOp(inp_qnt_node)
#inp_qnt.set_nodeattr("PE", 49)

os.makedirs(os.path.dirname(file_path+'/set_folding_factors'+file_name), exist_ok=True)
model.save(file_path+'/set_folding_factors'+file_name)
showInNetron(file_path+'/set_folding_factors'+file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/set_folding_factors/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [13]:
model = ModelWrapper(file_path+'/set_folding_factors'+file_name)
model = model.transform(GiveUniqueNodeNames())

In [3]:
from finn.analysis.fpgadataflow.hls_synth_res_estimation import hls_synth_res_estimation
resource_estimates = hls_synth_res_estimation(model)
print(resource_estimates)

{}


In [4]:
## Default builder flow for resource estimations. From advanced_builder_settings tutorial.
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

estimates_output_dir = file_path+"/output_estimates_only"

# Delete previous run results if exist.
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")

cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 10000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

Previous run results deleted!


In [5]:
%%time
build.build_dataflow_cfg(onnx_model, cfg_estimates);

TypeError: can only concatenate str (not "ModelProto") to str

In [ ]:
## Synthesizing into Vitis - will take a while.
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(file_path+'/set_folding_factors'+file_name)
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

In [ ]:
## Generate PYNQ driver from accelerator.
from finn.transformation.fpgadataflow.make_pynq_driver import MakePYNQDriver
model = model.transform(MakePYNQDriver("zynq-iodma"))
model.save(build_dir+'/post_synthesis'+file_name)
showInNetron(build_dir+'/post_synthesis'+file_name)

In [ ]:
model = ModelWrapper(build_dir+'/post_synthesis'+file_name)
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")
showInNetron(postsynth_layers)

In [ ]:
model = ModelWrapper(postsynth_layers)
model.model.metadata_props

In [ ]:
model = ModelWrapper(build_dir+'/post_synthesis'+file_name)
model.model.metadata_props

In [ ]:
! ls {model.get_metadata_prop("vivado_pynq_proj")} # Directory of synthesized project.